In [3]:
from Bio import SeqIO
from Bio.SeqUtils.IsoelectricPoint import IsoelectricPoint as IP
from Bio.SeqUtils.ProtParam import ProteinAnalysis as PA
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import os
from collections import defaultdict
import pandas as pd
import ipywidgets as widgets
from ipywidgets import *

In [26]:
def listtostr(strlist):
    output = ""
    for word in strlist:
        output += word + "_"
    return(output[:len(output)-1])
def simplifyname(orgname):
    firstname = [orgname.split("_")[0][0]]
    lastname = orgname.split("_")[1:]
    fullname = firstname + lastname
    return(listtostr(fullname))

In [71]:
#Generate a file path for the correct .fasta file
input_filename = input("Enter File Name")
datafile = os.path.join('data', input_filename)
organism_name = input_filename.replace(".faa","")

Enter File Name Yersinia_pestis[632].faa


In [72]:
data_folder = os.path.join('data', organism_name +"_affinity_chromatography")

if not os.path.isdir(data_folder):
    os.mkdir(data_folder)

In [73]:
def getHistPercent(protstring):
    Hcount = 0
    for AA in protstring:
        if AA == "H":
            Hcount += 1
    return(Hcount/len(protstring))

In [74]:
def findHistChains(protstring):
    chainDict = {}
    chainlen = 1
    lastAA = ""
    for AA in protstring:
        if AA == "H":
            if lastAA == "H":
                chainlen += 1
        elif chainlen > 1 and lastAA == "H":
            #print(f"Chain found, current length {chainlen}")
            if str(chainlen) in chainDict:
                chainDict[str(chainlen)] = chainDict[str(chainlen)] + 1
            else:
                chainDict[str(chainlen)] = 1
            chainlen = 1
        lastAA = AA
    if chainlen > 1:
        if str(chainlen) in chainDict:
            chainDict[str(chainlen)] = chainDict[str(chainlen)] + 1
        else:
            chainDict[str(chainlen)] = 1
    return(chainDict)

In [75]:
def HistScore(protstring):
    last4 = ""
    last8 = ""
    CD4 = 0
    CD8 = 0
    score = 0
    for AA in protstring:
        #iterate last4 & last8
        if len(last4) < 4:
            last4 += AA
        else:
            last4 = last4[1:] + AA
        if len(last8) < 8:
            last8 += AA
        else:
            last8 = last8[1:] + AA
        
        #scan for non-chain H groupings 
        if getHistPercent(last4) >= 0.5 and CD4 < 1:
            score += 2
            CD4 = 4
        else:
            CD4 -= 1
        if getHistPercent(last8) >= 0.375 and CD8 < 1:
            score += 6
            CD8 = 8
        else:
            CD8 -= 1
    
    #scan for chains
    histChains = findHistChains(protstring)
    for key in histChains:
        score += histChains[key] * (int(key) ** 2)
    return(score)

In [76]:
unfiltered = []
wash = []
fractions = []
with open(datafile) as protfile:
    for record in SeqIO.parse(protfile,"fasta"):
        unfiltered.append(record)
        protparams = PA(str(record.seq))
        if HistScore(record.seq) < 8:
            wash.append(record)
        else:
            fractions.append(record)    

data_name = "Affinity_chromatography_" + simplifyname(organism_name)
SeqIO.write(unfiltered,os.path.join(data_folder,data_name+"_total.faa"),"fasta")
SeqIO.write(wash,os.path.join(data_folder,data_name+"_wash.faa"),"fasta")
SeqIO.write(fractions,os.path.join(data_folder,data_name+"_fractions.faa"),"fasta")


649

In [77]:
datadict = {
    'sequence' : [],
    'length' : [],
    'isoelectric_point' : [],
    'hist_score' : [],
    'max_chain_length': [],
    'ID' : [],
    'description' : []
}

with open(os.path.join(data_folder,data_name+"_fractions.faa"),"r") as prot_file:
    for record in SeqIO.parse(prot_file,"fasta"):
        sequence = str(record.seq)  #It is not strictly necessary to record the sequence as a string, but it is easier to work with and displays better. (Ex:len() does not work on Seq objects)
        parameters = PA(sequence)
        datadict['sequence'].append(sequence)
        datadict['length'].append(len(sequence))
        datadict['isoelectric_point'].append(parameters.isoelectric_point())
        datadict['hist_score'].append(HistScore(sequence))
        datadict['max_chain_length'].append(max(findHistChains(sequence).keys(),default = '0'))
        datadict['ID'].append(record.id)
        datadict['description'].append(record.description)

df = pd.DataFrame.from_dict(datadict)
df.sort_values('hist_score',inplace = True)
df.reset_index(drop=True, inplace=True)
df

,sequence,length,isoelectric_point,hist_score,max_chain_length,ID,description
0,MRNEDFKSFMAQEYNEAFDSIGMMIETPNVGDQITLKDPRRQINLS...,435,8.100082,8,0,tr|Q0W9Z3|Q0W9Z3_YERPE,tr|Q0W9Z3|Q0W9Z3_YERPE Putative uncharacterize...
1,MHMTDFTISPKAENVWLESWLDLSSEEKREMDHIEQDEQCDARFFH...,95,4.393506,8,0,tr|Q9RID7|Q9RID7_YERPE,tr|Q9RID7|Q9RID7_YERPE Putative uncharacterize...
2,MSLSLLGIYLYNLSRDNLRPEEYERIVSAYAAWTRVCREYEFNDGY...,77,6.408164,8,2,tr|Q7CFZ1|Q7CFZ1_YERPE,tr|Q7CFZ1|Q7CFZ1_YERPE Putative uncharacterize...
3,MQRGIVWIVDDDSSIRWVLERALTGAGLNCVAFDSGNQVLDALATR...,470,6.079465,8,0,tr|Q0WKS0|Q0WKS0_YERPE,tr|Q0WKS0|Q0WKS0_YERPE Nitrogen regulation pro...
4,MSTSLPTQLCANTPALTIHDNRGLAIRTLAYNRRDHNETVDELISR...,952,7.385113,8,2,tr|Q7CL71|Q7CL71_YERPE,tr|Q7CL71|Q7CL71_YERPE Putative insecticidal t...
...,...,...,...,...,...,...,...
644,MKVTKDLVVSLAYQVRTEDGVLVDESPVSAPLDYLHGHGSLIAGLE...,195,4.719876,47,3,sp|Q7CFU4|SLYD_YERPE,sp|Q7CFU4|SLYD_YERPE FKBP-type peptidyl-prolyl...
645,MLIPSKLSRPVRLQNTVVRDRLLVKLSSAANYRLTLINCPAGYGKT...,903,5.990000,48,2,sp|Q8ZJI2|MALT_YERPE,sp|Q8ZJI2|MALT_YERPE HTH-type transcriptional ...
646,MNILLISQCHKRALNETRRILDQFAERKGDRTWQTAITQEGLNTLR...,1095,8.878602,52,2,tr|Q7CJ41|Q7CJ41_YERPE,tr|Q7CJ41|Q7CJ41_YERPE Predicted helicases OS=...
647,MAVSTIFSQDSNSKRLLIAFAITTLFMVTEAIGGWLSGSLALLADA...,312,6.431525,54,4,sp|Q8ZGY6|ZITB_YERPE,sp|Q8ZGY6|ZITB_YERPE Zinc transporter zitB OS=...


In [78]:
def param_of_interest(protdata):
    output = bool(HistScore(protdata) >= 4)
    return(output)

nfractions = 7
fraclen = round(len(df)/nfractions)
seqofinterest = "HHHHHH"
seqhits = {}
noise = 0.10
for n in range(nfractions - 1):
    fracrecords = []
    #Generate min/max range values
    fuzzymin = int(round(n*fraclen-noise*fraclen,0))
    fuzzymax = int(round((n+1)*fraclen + noise*fraclen,0))
    if fuzzymin < 0:
        fuzzymin = 0
    if fuzzymax > len(df):
        fuzzymax = len(df)
    tempdf = df.iloc[fuzzymin:fuzzymax]
    for index  in tempdf.index:
        record = SeqRecord(Seq(tempdf['sequence'][index]),id=str(tempdf["ID"][index]),description=tempdf["description"][index])
        fracrecords.append(record)
        if param_of_interest(tempdf['sequence'][index]) == True:
            if n+1 in seqhits:
                seqhits[n+1][0] += 1
                seqhits[n+1][1].append(index)
            else:
                seqhits[n+1] = [1,[index]]
    SeqIO.write(fracrecords,os.path.join(data_folder,data_name+"_fraction"+ str(n+1) + ".faa"),"fasta")

tempdf = df.iloc[int(round((nfractions-1)*fraclen-noise*fraclen,0)):]
remainderrecord = []
for index  in tempdf.index:
        record = SeqRecord(Seq(tempdf['sequence'][index]),id=str(tempdf["ID"][index]),description=tempdf["description"][index])
        remainderrecord.append(record)
        if param_of_interest(tempdf['sequence'][index]) == True:
            if nfractions in seqhits:
                seqhits[nfractions][0] += 1
                seqhits[nfractions][1].append(index)
            else:
                seqhits[nfractions] = [1,[index]]
SeqIO.write(remainderrecord,os.path.join(data_folder,data_name+"_fraction"+ str(nfractions) + ".faa"),"fasta")
tempdf

,sequence,length,isoelectric_point,hist_score,max_chain_length,ID,description
549,MVAKLEQLIAQTILQGFDAQYGRFLEVTAGAQHRFEQADWHAVQQA...,575,6.018647,18,2,sp|Q8ZAR7|ACEK_YERPE,sp|Q8ZAR7|ACEK_YERPE Isocitrate dehydrogenase ...
550,MDYDLIVIGSGSVGSAAGYYASQAGLNVLMIDSAMPPHQAGSHHGE...,371,5.278831,18,2,sp|P58526|MTOX_YERPE,sp|P58526|MTOX_YERPE N-methyl-L-tryptophan oxi...
551,MATAHHTDQLHSLQTHGIHCGETTIPSQGDELPAYIAKPDQHTGPY...,267,5.934639,18,2,sp|Q8ZAL4|DLHH_YERPE,sp|Q8ZAL4|DLHH_YERPE Putative carboxymethylene...
552,MRNHASGGATTIIKRANRKKHHVRHSGAWKVAFADFTLAMMALFMV...,303,6.772502,18,2,tr|Q74RH7|Q74RH7_YERPE,tr|Q74RH7|Q74RH7_YERPE Chemotaxis MotB protein...
553,MRQQLAGTELVFTQPFNAAERQVLPDKAIEFLTELVVKFSESRNQL...,532,5.964366,18,2,tr|Q0WAT2|Q0WAT2_YERPE,tr|Q0WAT2|Q0WAT2_YERPE Malate synthase OS=Yers...
...,...,...,...,...,...,...,...
644,MKVTKDLVVSLAYQVRTEDGVLVDESPVSAPLDYLHGHGSLIAGLE...,195,4.719876,47,3,sp|Q7CFU4|SLYD_YERPE,sp|Q7CFU4|SLYD_YERPE FKBP-type peptidyl-prolyl...
645,MLIPSKLSRPVRLQNTVVRDRLLVKLSSAANYRLTLINCPAGYGKT...,903,5.990000,48,2,sp|Q8ZJI2|MALT_YERPE,sp|Q8ZJI2|MALT_YERPE HTH-type transcriptional ...
646,MNILLISQCHKRALNETRRILDQFAERKGDRTWQTAITQEGLNTLR...,1095,8.878602,52,2,tr|Q7CJ41|Q7CJ41_YERPE,tr|Q7CJ41|Q7CJ41_YERPE Predicted helicases OS=...
647,MAVSTIFSQDSNSKRLLIAFAITTLFMVTEAIGGWLSGSLALLADA...,312,6.431525,54,4,sp|Q8ZGY6|ZITB_YERPE,sp|Q8ZGY6|ZITB_YERPE Zinc transporter zitB OS=...


In [60]:
seqhits
for key in seqhits:
    print(f'Hits in fraction {key}')
    for hit in seqhits[key][1]:
        print(f'\tHit on index {hit}')

Hits in fraction 1
	Hit on index 0
	Hit on index 1
Hits in fraction 2
	Hit on index 2
	Hit on index 3
Hits in fraction 3
	Hit on index 4
	Hit on index 5
Hits in fraction 4
	Hit on index 6
	Hit on index 7
Hits in fraction 5
	Hit on index 8
	Hit on index 9
Hits in fraction 6
	Hit on index 10


In [48]:
df.iloc[10]['sequence']

'MNKYDLIERMNTRFAELEVTLHQLHQQLDDLPLIAARVFSLPEIEKGTEHQPIEQITVNITEGEHAKKLGLQHFQRLFLHHQGQHVSSKAALRLPGVLCFSVTDKELIECQDIIKKTNQLKAELEHIITVESGLPSEQRFEFVHTHLHGLITLNTYRTITPLINPSSVRFGWANKHIIKNVTREDILLQLEKSLNAGRAVPPFTREQWRELISLEINDVQRLPEKTRLKIKRPVKVQPIARVWYQEQQKQVQHPCPMPLIAFCQHQLGAELPKLGELTDYDVKHIKHKYKPDAKPLRLLVPRLHLYVELEP'

In [64]:
data_folder

'data\\Y_pestis[632]_IonEx+SizeEx_affinity_chromatography'